<a href="https://colab.research.google.com/github/TanushriPatil/NLP-Disaster-Tweets-Prediction/blob/main/NLP_Disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install emoji

     |████████████████████████████████| 133kB 6.8MB/s 


In [2]:
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')

import string
import emoji
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [4]:
train_df = pd.read_csv("/content/gdrive/MyDrive/Data Science/NLP_disaster_tweets/train.csv")
test_df = pd.read_csv("/content/gdrive/MyDrive/Data Science/NLP_disaster_tweets/test.csv")


In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
test_df.head()
print(test_df.shape)

(3263, 4)


In [7]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
X = train_df['text']
Y = train_df['target']

## Data Pre- processing
#### This involves:
#### 1. Cleaning the data (removing emojis, hyperlink, punctuation, etc) 
#### 2. Converting to lowercase
#### 3. Removing stopwords
#### 4. Tokenization  

In [9]:
punc = string.punctuation

In [10]:
def data_cleaning(txt):
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    txt = re.sub(r'#', '', txt)
    txt = re.sub(emoji.get_emoji_regexp(), r"", txt)
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+','',txt)
    txt = re.sub(r"https?://\S+|www\.\S+","",txt)
    txt = re.sub(r"<.*?>","",txt)
    return txt  

def remove_punct(txt):
    #symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in punc:
        txt = np.char.replace(txt, i, '')
    return txt
        
def lower_case(txt):
    return str.lower(txt)


In [11]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
def remove_stopwords(txt):
    txt = [word for word in txt.split() if word not in stop_words]
    return " ".join(txt)
        

In [12]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stemming_process(txt):
  txt = [stemmer.stem(word) for word in txt]
  txt=''.join(txt)
  return txt

In [13]:
train_df.text = train_df.text.apply(data_cleaning)
train_df.text = train_df.text.apply(remove_punct)
train_df.text = train_df.text.apply(lower_case)
train_df.text = train_df.text.apply(remove_stopwords)
#train_df.text = train_df.text.apply(tokenize_word)
#train_df.text = train_df.text.apply(stemming_process)

test_df.text = test_df.text.apply(data_cleaning)
test_df.text = test_df.text.apply(remove_punct)
test_df.text = test_df.text.apply(lower_case)
test_df.text = test_df.text.apply(remove_stopwords)
#test_df.text = test_df.text.apply(tokenize_word)
#test_df.text = test_df.text.apply(stemming_process)

In [14]:
cleaned_data = []
cleaned_data = train_df.text.to_list()
cleaned_data
test_cleaned_data = []
test_cleaned_data = test_df.text.to_list()

In [15]:
train_df = train_df[['text', 'target']]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500, stop_words=["\x89û"])
X_fin = cv.fit_transform(cleaned_data).toarray()
print(X_fin.shape)
fin_test = cv.fit_transform(test_cleaned_data).toarray()
print(fin_test.shape)

(7613, 1500)
(3263, 1500)


In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_fin, Y, test_size=0.2)
model = MultinomialNB()

In [18]:
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
cf = classification_report(y_test, y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.79      0.87      0.83       874
           1       0.79      0.68      0.73       649

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [21]:
model.fit(X_fin, Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
y_pred2 = model.predict(fin_test)

In [23]:
sub_df = pd.DataFrame(y_pred2.tolist())
# ff = y_pred2.tolist()
sub_df.rename(columns={0: "target_nb"}, inplace=True)
sub_df
#sub_df

,target_nb
0,0
1,1
2,0
3,1
4,0
...,...
3258,1
3259,0
3260,0
3261,1


Prediction using RNN

In [24]:
#pip install tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [25]:
#### Vocabulary size
voc_size = 9000

In [26]:
onehot_rep = [one_hot(words, voc_size) for words in cleaned_data]
print(onehot_rep[1])
onehot_rep_test = [one_hot(words, voc_size) for words in test_cleaned_data]
print(onehot_rep_test[1])

[2513, 95, 2334, 5608, 2486, 3153, 7405]
[7845, 1513, 7702, 3479, 4955, 7358, 4336]


Embedding Representation

In [27]:
sent_length =25
embedded_docs = pad_sequences(onehot_rep, padding="pre", maxlen = sent_length)
print(embedded_docs) 

embedded_docs_test = pad_sequences(onehot_rep_test, padding="pre", maxlen=sent_length )

[[   0    0    0 ... 5109 6258  296]
 [   0    0    0 ... 2486 3153 7405]
 [   0    0    0 ... 1941 8144 8191]
 ...
 [   0    0    0 ... 7414 7809 6439]
 [   0    0    0 ... 8015 4383 6707]
 [   0    0    0 ...  348 2564 1475]]


In [28]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 2513,   95, 2334, 5608,
       2486, 3153, 7405], dtype=int32)

In [29]:
#### creating a model
feature_vector = 100
model = Sequential()
model.add(Embedding(voc_size, feature_vector, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.add(Dropout(0.3))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 100)           900000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
_________________________________________________________________
dropout (Dropout)            (None, 1)                 0         
Total params: 980,501
Trainable params: 980,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
len(embedded_docs),Y.shape

(7613, (7613,))

In [31]:
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

X_train, X_test, y_train, y_test = train_test_split(X_final, Y_final, test_size=0.2)



##**Model** **Training**

In [32]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=264)

loss, accuracy = model.evaluate(X_train, y_train, verbose = 0)
print("Training accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose = 0)
print("Testing accuracy: {:.4f}".format(accuracy))

Epoch 1/10
24/24 [==============================] - 6s 161ms/step - loss: 2.5452 - accuracy: 0.5335 - val_loss: 0.6721 - val_accuracy: 0.5666
Epoch 2/10
24/24 [==============================] - 3s 139ms/step - loss: 2.5098 - accuracy: 0.6824 - val_loss: 0.6179 - val_accuracy: 0.6507
Epoch 3/10
24/24 [==============================] - 3s 135ms/step - loss: 2.2467 - accuracy: 0.7502 - val_loss: 0.5409 - val_accuracy: 0.7538
Epoch 4/10
24/24 [==============================] - 3s 133ms/step - loss: 2.2370 - accuracy: 0.7778 - val_loss: 0.5269 - val_accuracy: 0.7669
Epoch 5/10
24/24 [==============================] - 3s 135ms/step - loss: 2.0326 - accuracy: 0.8082 - val_loss: 0.5387 - val_accuracy: 0.7525
Epoch 6/10
24/24 [==============================] - 3s 134ms/step - loss: 2.1180 - accuracy: 0.8186 - val_loss: 0.5574 - val_accuracy: 0.7505
Epoch 7/10
24/24 [==============================] - 3s 134ms/step - loss: 2.1179 - accuracy: 0.8268 - val_loss: 0.6006 - val_accuracy: 0.7531
Epoch 

## **Prediction**

In [49]:
#y_pred = (model.predict(embedded_docs_test) > 0.5).astype("int32")
y_pred = model.predict_classes(X_test)
#y_pred = model.predict_classes(embedded_docs_test)
import warnings
warnings.filterwarnings("ignore")

In [34]:
y_pred

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [35]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[771,  92],
       [300, 360]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7426132632961261

In [37]:
model.fit(X_final, Y_final, epochs=10, verbose=1 )

Epoch 1/10
238/238 [==============================] - 8s 35ms/step - loss: 2.2687 - accuracy: 0.7915
Epoch 2/10
238/238 [==============================] - 8s 35ms/step - loss: 2.2613 - accuracy: 0.8040
Epoch 3/10
238/238 [==============================] - 8s 35ms/step - loss: 2.1370 - accuracy: 0.8269
Epoch 4/10
238/238 [==============================] - 8s 35ms/step - loss: 2.1465 - accuracy: 0.8291
Epoch 5/10
238/238 [==============================] - 8s 35ms/step - loss: 2.1707 - accuracy: 0.8333
Epoch 6/10
238/238 [==============================] - 8s 35ms/step - loss: 2.0773 - accuracy: 0.8436
Epoch 7/10
238/238 [==============================] - 8s 35ms/step - loss: 2.0399 - accuracy: 0.8512
Epoch 8/10
238/238 [==============================] - 8s 35ms/step - loss: 2.0269 - accuracy: 0.8528
Epoch 9/10
238/238 [==============================] - 8s 35ms/step - loss: 2.1121 - accuracy: 0.8493
Epoch 10/10
238/238 [==============================] - 8s 35ms/step - loss: 2.1348 - accura

In [45]:
loss, accuracy = model.evaluate(X_final, Y_final, verbose = 0)
print("Training accuracy: {:.4f}".format(accuracy))

Training accuracy: 0.9744


In [50]:
y_pred = model.predict_classes(embedded_docs_test)
y_pred
warnings.filterwarnings("ignore")

In [40]:
embedded_docs_test.shape

(3263, 25)

In [47]:
#sub_df = pd.DataFrame(y_pred2.tolist())
# ff = y_pred2.tolist()
#sub_df.rename(columns={0: "target_nb"}, inplace=True)
sub_df['target_rnn'] = y_pred.tolist()
sub_df

,target_nb,target_rnn
0,0,[1]
1,1,[0]
2,0,[1]
3,1,[1]
4,0,[0]
...,...,...
3258,1,[0]
3259,0,[1]
3260,0,[1]
3261,1,[1]


In [43]:
sub_df.to_csv("prediction.csv")